In [1]:
from pathlib import Path
import geopandas as gpd
import folium 
from shapely.geometry import LineString, Polygon, box, shape
import rasterio
import os
import matplotlib.pyplot as plt
import pandas as pd
import rasterio.features

In [2]:
import ra2ce.network.networks_utils as nut
from ra2ce.network.network_config_data.enums.network_type_enum import NetworkTypeEnum
from ra2ce.network.network_config_data.enums.road_type_enum import RoadTypeEnum
from ra2ce.network.network_config_data.network_config_data import (NetworkConfigData,NetworkSection)
from ra2ce.network.exporters.geodataframe_network_exporter import GeoDataFrameNetworkExporter
from ra2ce.network.exporters.multi_graph_network_exporter import MultiGraphNetworkExporter
from ra2ce.network.network_config_data.enums.aggregate_wl_enum import AggregateWlEnum
from ra2ce.network.network_config_data.enums.source_enum import SourceEnum
from ra2ce.network.network_config_data.network_config_data import (NetworkConfigData,NetworkSection,HazardSection, CleanupSection)
from ra2ce.ra2ce_handler import Ra2ceHandler
from ra2ce.analysis.damages.damages import AnalysisSectionDamages
from ra2ce.configuration.ra2ce_enum_base import Ra2ceEnumBase
from ra2ce.analysis.analysis_config_data.enums.analysis_damages_enum import AnalysisDamagesEnum
from ra2ce.analysis.analysis_config_data.enums.event_type_enum import EventTypeEnum
from ra2ce.analysis.analysis_config_data.enums.damage_curve_enum import DamageCurveEnum
from ra2ce.analysis.analysis_config_data.analysis_config_data import AnalysisConfigData
from ra2ce.network.network_wrappers.osm_network_wrapper.osm_network_wrapper import OsmNetworkWrapper

c:\Users\meije_le\AppData\Local\miniforge3\envs\ra2ce_env_brs\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:
root_dir = Path(r"P:\bovenregionale-stresstest-hwn\Scripts\RA2CE_Model\data\Test_thresholds")

static_path = root_dir.joinpath("static") #within your root directory there should be a 'static' folder
network_path = root_dir.joinpath("network") #within the static folder there should be a 'network' folder
hazard_path = root_dir.joinpath("hazard") #within the static folder there should be a 'hazard' folder
output_path = root_dir.joinpath("output")
assert root_dir.exists(), "root_dir not found."

In [4]:
hazard_map_path = hazard_path.joinpath("HAA_scenario_113816_max_waterdepth.tiff")  # Put the name of your hazard map here

In [11]:
#First we define which roads we will want to download from OSM to create a network with
network_section = NetworkSection(
    source= SourceEnum.SHAPEFILE,       
    primary_file = [network_path.joinpath("segmented_network_Complete.gpkg")], 
    file_id = "ID",
    link_type_column="highway",
    save_gpkg=True
    #attributes_to_exclude_in_simplification=['bridge', 'tunnel'], 
)
#We input the hazard map(s)
hazard = HazardSection(
    hazard_map=[hazard_map_path],
    aggregate_wl = AggregateWlEnum.MAX,
    hazard_crs = "EPSG:28992"
)

#cleanup_section = CleanupSection(
#    segmentation_length=100
#)

#pass the specified sections as arguments for configuration

network_config_data = NetworkConfigData(
    root_path=root_dir,
    static_path=static_path,
    output_path=output_path,
    network=network_section,
    hazard=hazard,
    #cleanup=cleanup_section,
    )


In [12]:
section_damage = [AnalysisSectionDamages(
    name='HZ_damage',
    analysis=AnalysisDamagesEnum.DAMAGES,
    event_type=EventTypeEnum.EVENT,
    damage_curve=DamageCurveEnum.HZ,
    save_gpkg=True,
    save_csv=True,
)]

analysis_config_data = AnalysisConfigData(analyses=section_damage, root_path=root_dir, output_path=output_path)
analysis_config_data.input_path = root_dir.joinpath("input_data")

In [13]:
with open(r"P:\bovenregionale-stresstest-hwn\Scripts\RA2CE_Model\data\Test_thresholds\static\output_graph\test.txt", "w") as f:
    f.write("test")


### Run RA2CE.

In [14]:
handler = Ra2ceHandler.from_config(network_config_data, None)
handler.configure()
handler.run_analysis()

100%|██████████| 20956/20956 [00:00<00:00, 339376.64it/s]
2025-08-05 05:37:34 PM - [network_graph_simplificator.py:139] - root - ERROR - Could not find the simple ID belonging to complex ID (2708, 2709, 0); value set to None. Full error: 3441
2025-08-05 05:37:34 PM - [network_graph_simplificator.py:139] - root - ERROR - Could not find the simple ID belonging to complex ID (2708, 2709, 0); value set to None. Full error: 3441
2025-08-05 05:37:34 PM - [network_graph_simplificator.py:139] - root - ERROR - Could not find the simple ID belonging to complex ID (2708, 2709, 0); value set to None. Full error: 3441
2025-08-05 05:37:34 PM - [network_graph_simplificator.py:139] - root - ERROR - Could not find the simple ID belonging to complex ID (2708, 9341, 0); value set to None. Full error: 3442
2025-08-05 05:37:34 PM - [network_graph_simplificator.py:139] - root - ERROR - Could not find the simple ID belonging to complex ID (2708, 9341, 0); value set to None. Full error: 3442
2025-08-05 05:37:

CPLE_AppDefinedError: b'sqlite3_exec(CREATE TABLE gpkg_extensions (table_name TEXT,column_name TEXT,extension_name TEXT NOT NULL,definition TEXT NOT NULL,scope TEXT NOT NULL,CONSTRAINT ge_tce UNIQUE (table_name, column_name, extension_name))) failed: disk I/O error'

Exception ignored in: 'fiona._shim.gdal_flush_cache'
Traceback (most recent call last):
  File "fiona\_err.pyx", line 201, in fiona._err.GDALErrCtxManager.__exit__
fiona._err.CPLE_AppDefinedError: b'sqlite3_exec(CREATE TABLE gpkg_extensions (table_name TEXT,column_name TEXT,extension_name TEXT NOT NULL,definition TEXT NOT NULL,scope TEXT NOT NULL,CONSTRAINT ge_tce UNIQUE (table_name, column_name, extension_name))) failed: disk I/O error'


TypeError: 'HazardFiles' object is not subscriptable